In [ ]:
# !pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00


In [ ]:
# from datasets import load_dataset

# Tải tập dữ liệu từ Hugging Face's Hub
sft_ds_name = 'carperai/openai_summarize_tldr'
sft_ds = load_dataset(sft_ds_name)

# Tiếp tục sử dụng tập dữ liệu đã tải
sft_train = sft_ds['train']
sft_valid = sft_ds['valid']
sft_test = sft_ds['test']

def formatting_func(example):
    text = f"### Text: {example['prompt']}\n### Summary: {example['label']}"
    return text

# Demo formatting
for example in sft_train:
    print(formatting_func(example))
    break  # Đảm bảo chỉ in một ví dụ để demo



### Text: SUBREDDIT: r/relationships
TITLE: I (f/22) have to figure out if I want to still know these girls or not and would hate to sound insulting
POST: Not sure if this belongs here but it's worth a try. 

Backstory:
When I (f/22) went through my first real breakup 2 years ago because he needed space after a year of dating roand  it effected me more than I thought. It was a horrible time in my life due to living with my mother and finally having the chance to cut her out of my life. I can admit because of it was an emotional wreck and this guy was stable and didn't know how to deal with me. We ended by him avoiding for a month or so after going to a festival with my friends. When I think back I wish he just ended. So after he ended it added my depression I suffered but my friends helped me through it and I got rid of everything from him along with cutting contact. 

Now: Its been almost 3 years now and I've gotten better after counselling and mild anti depressants. My mother has bee

In [ ]:
# !pip install trl
# !pip install peft
# !pip install rouge_score
from transformers import AutoTokenizer
import torch
from trl import ModelConfig, get_quantization_config, get_kbit_device_map
from peft import LoraConfig, PeftConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training


model_config = ModelConfig(
    model_name_or_path='facebook/opt-350m'
)

torch_dtype = (
    model_config.torch_dtype
    if model_config.torch_dtype in ["auto", None]
    else getattr(torch, model_config.torch_dtype)
)
quantization_config = get_quantization_config(model_config)
model_kwargs = dict(
    revision=model_config.model_revision,
    trust_remote_code=model_config.trust_remote_code,
    attn_implementation=model_config.attn_implementation,
    torch_dtype=torch_dtype,
    use_cache=False,
    device_map=get_kbit_device_map() if quantization_config is not None else None,
    quantization_config=quantization_config,
)
tokenizer = AutoTokenizer.from_pretrained(model_config.model_name_or_path, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

# lora
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)


1.3 Metric


In [ ]:
# !pip install evaluate
import evaluate

rouge = evaluate.load("rouge")

def compute_metrics(eval_preds):
  if isinstance(eval_preds, tuple):
    eval_preds = eval_preds[0]
  label_ids = eval_preds.label_ids
  pred_ids = eval_preds.predictions
  pred_str = tokenizer.batch_decode(pred_ids, skip_special_token=True)
  label_str = tokenizer.batch_decode(label_ids, skip_special_token=True)
  result = rouge.compute(predictions = pred_str, references = label_str)


In [ ]:
# Giới hạn số lượng dữ liệu bạn muốn lấy
num_desired_train_data = 20000

# Lấy 20,000 dữ liệu từ tập huấn luyện
sft_train = sft_train[:19999]
sft_test = sft_test[:120]
sft_valid = sft_valid[:120]


TypeError: unhashable type: 'slice'

1.4 trainer

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

num_epochs = 1
train_args = TrainingArguments(
    output_dir ='./save_model',
    evaluation_strategy="epoch",
    save_strategy='epoch',
    per_device_train_batch_size= 4,
    per_device_eval_batch_size= 4,
    adam_beta1=0.9,
    adam_beta2 = 0.95,
    num_train_epochs= num_epochs,
    load_best_model_at_end= True,
)

max_input_length = 512
trainer = SFTTrainer(
    model = model_config.model_name_or_path,
    model_init_kwargs = model_kwargs,
    args = train_args,
    train_dataset = sft_train,
    eval_dataset = sft_valid,
    max_seq_length=max_input_length,
    tokenizer=tokenizer,
    peft_config = peft_config,
    compute_metrics = compute_metrics,
    packing = True,
    formatting_func = formatting_func
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:165: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

ValueError: Error occurred while packing the dataset. Make sure that your dataset has enough samples to at least yield one packed sequence.